In [1]:
import pandas as pd
import gc

In [2]:
gc.collect() #memory
print("RB_reducing_txt.ipynb takes around 5 minutes to run")

RB_reducing_txt.ipynb takes around 5 minutes to run


In [3]:
new_column_names = [
    'beer_name', 'beer_id', 'brewery_name', 'brewery_id', 'style', 'abv', 'date',
    'user_name', 'user_id', 'appearance', 'aroma', 'palate', 'taste', 'overall',
    'rating', 'text'
]

output_file_path = 'minimizedData/RB_ratings_small.csv'

def parse_ratings_file(file_path, chunk_size=15_000_001, start=0, stop=200_000_000): #chunk is a mulitple of 17 since our text 'block' size is 17
    #initialise the csv
    pd.DataFrame(columns=new_column_names).drop(['user_name','abv','date','text'], axis=1).to_csv(
        output_file_path, index=False
    )

    current_block = []
    data = []
    i = 0
    k = 0 
    chunk = 0
    
    with open(file_path, 'r', encoding = 'utf-8') as file:
        for line in file:
            i += 1
            k += 1
            if i<start: continue
           
            if i+16>stop:break
           
            if i % 10_000_000 == 0: print('Processing line: ', i)

            line = line.strip()  #remove whitespace

            if not line:  #empty line signifies end of block
                data.append(current_block)
                current_block = []
                
            if k >= chunk_size:
                k = 0
                chunk += 1
                print('Processing chunk :', chunk)
                
                df = pd.DataFrame(data, columns=new_column_names)
                df = df.drop(['user_name', 'abv', 'date', 'text'], axis=1)
                df.to_csv(output_file_path, mode='a', index=False, header=False)
                data = []  
                gc.collect()  #manually invoke garbage collection
                               
            if ': ' in line:  
                key, value = line.split(': ', 1)
                current_block.append(value)

        if data: #save any leftover data at the end
            data.append(current_block)
            current_block = []
            print(k)
            k = 0
            chunk += 1 
            print('Processing chunk :', chunk)
            
            df = pd.DataFrame(data, columns=new_column_names)
            df = df.drop(['user_name', 'abv', 'date', 'text'], axis=1)
            df.dropna()
            df.to_csv(output_file_path, mode='a', index=False, header=False)
            data = []  
            gc.collect() 
            
    print('Done!')

In [4]:
#takes 4 mins
print("There are around 120 million lines of text")
parse_ratings_file('baseData/RateBeer/ratings.txt')

There are around 120 million lines of text


Processing line:  10000000


Processing chunk : 1


Processing line:  20000000


Processing line:  30000000
Processing chunk : 2


...

Processing chunk : 7


Processing line:  110000000


Processing line:  120000000
Processing chunk : 8


1075250
Processing chunk : 9


Done!


In [5]:
df = pd.read_csv('minimizedData/RB_ratings_small.csv')
df = df.drop(df.index[-1])
df.to_csv('minimizedData/RB_ratings_small.csv', index=False)
print("Saved 'minimizedData/RB_ratings_small.csv'")
#df = pd.read_csv('minimizedData/RB_ratings_small.csv')
#df.tail(2)

Saved 'minimizedData/RB_ratings_small.csv'


In [6]:
#df.head(2)

In [7]:
#df.sample(3)